# Utility Agent

This is a vaccume clearner agent with is a utility based agent which has a utility function which check if it will be closer to dirt if it made that move or another. Basically its happiness meter will be high if it is around many dirt particles and less if there are no dirt particles around it. So When it need to make a move it first sees its surrounding for dirt if present then goes there. If not then it has a vision of one block that is it will see that if it made a particular move will it find more dirt or no and then make the move which gives it the maximum happiness (that is max utility value).

In [1]:
import random

This is a room environment which either taken in a custom room as input or generates a room at random when given the dimensions. 

In [2]:
class room_environment:
    def __init__(self, n, m, room=None, dirt=None):
        self.m = m
        self.n = n
        self.room = []
        self.num_dirt = 0
        if(room == None):
            for i in range(self.n):
                row = []
                for j in range(self.m):
                    row.append(0)
                self.room.append(row)
                
            num_walls = random.choice([random.randint(self.n//2, self.n-1), random.randint(self.m//2, self.m-1)])
            for i in range(num_walls):
                wall_x = random.randint(0, self.m-1)
                wall_y = random.randint(0, self.n-1)
                self.room[wall_x][wall_y] = -1
                
            num_dirt = random.choice([random.randint(1, self.n//2), random.randint(1, self.m//2)])
            self.num_dirt = num_dirt
            for i in range(num_dirt):
                dirt_x = random.randint(0, self.m-1)
                dirt_y = random.randint(0, self.n-1)
                self.room[dirt_x][dirt_y] = 1
        else:
            self.room = room
            self.num_dirt = dirt
    
    def get_state(self, pos):
        if(pos[0] >= self.m or pos[0] < 0 or pos[1] >= self.n or pos[1] < 0):
            return -1
        return self.room[pos[1]][pos[0]]
    
    def pickup_dirt(self, pos):
        if(self.room[pos[1]][pos[0]] == 1):
            self.room[pos[1]][pos[0]] = 0
    
    def dirt_count(self):
        return self.num_dirt
    
    def print_room(self):
        for i in range(self.n):
            for j in range(self.m):
                print(self.room[i][j], end=' ')
            print()

In [3]:
room = room_environment(10, 10)

In [4]:
room.print_room()

0 0 0 -1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 -1 0 1 0 0 
0 0 0 -1 0 0 -1 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 -1 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 


In [5]:
room.dirt_count()

1

In [6]:
room.get_state((6, 3)) #get given positions state value

0

In [7]:
room.pickup_dirt((6, 3)) #If dirt is present in the given state then pick it up
room.get_state((6, 3)) #After pickup the state becomes empty

0

### Utility Agent

In [8]:
class cleaner_agent:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.limit = 0
        self.dirt_collected = 0
        self.prev_move = -1
    
    def get_surroundings(self, x, y, room):          
        surroundings = [room.get_state([x+1, y]),
                        room.get_state([x, y+1]),
                        room.get_state([x-1, y]),
                        room.get_state([x, y-1])]
        return surroundings
    
    def make_move(self, direction, room):
        if(direction == 0):
            if(self.x < room.m):
                self.x += 1
        elif(direction == 1):
            if(self.y < room.n):
                self.y += 1
        elif(direction == 2):
            if(self.x > 0):
                self.x -= 1
        else:
            if(self.y > 0):
                self.y -= 1
            
    def get_pos(self):
        print("I am at:", self.x, self.y, " and picked up: ", self.dirt_collected, " dirt.")
    
    def move(self, room):
        if(self.limit >= room.m*room.n):
            print("Moves up...")
            return False
        elif(self.dirt_collected == room.dirt_count()):
            print("Collected All the Dirst, Job Done...")
            return False
        else:
            surroundings = self.get_surroundings(self.x, self.y, room)
            dirt_around = []
            for i in range(len(surroundings)):
                if surroundings[i] == 1:
                    dirt_around.append(i)
            if(dirt_around):
                rand_move = random.choice(dirt_around)
                self.make_move(rand_move, room)
                room.pickup_dirt([self.x, self.y])
                self.limit -= room.n
                self.prev_move = rand_move
                self.dirt_collected += 1
                self.limit += 1
                return True
            else:
                possible_scores = [0, 0, 0, 0]
                for i in range(self.x+1, room.m):
                    if(room.get_state([i, self.y]) == 1):
                        possible_scores[0] += 1
                    elif(room.get_state([i, self.y]) == -1):
                        break
                for j in range(self.y+1, room.n):
                    if(room.get_state([self.x, j]) == 1):
                        possible_scores[1] += 1
                    elif(room.get_state([self.x, j]) == -1):
                        break
                for i in range(self.x-1, -1, -1):
                    if(room.get_state([i, self.y]) == 1):
                        possible_scores[2] += 1
                    elif(room.get_state([i, self.y]) == -1):
                        break
                for j in range(self.y-1, -1, -1):
                    if(room.get_state([self.x, j]) == 1):
                        possible_scores[3] += 1
                    elif(room.get_state([self.x, j]) == -1):
                        break

                if(max(possible_scores) == 0):
                    moves = [0, 1, 2, 3]
                    if(self.prev_move == 0):
                        moves.remove(2)
                    elif(self.prev_move == 1):
                        moves.remove(3)
                    elif(self.prev_move == 2):
                        moves.remove(0)
                    elif(self.prev_move == 3):
                        moves.remove(1)
                    best_move = random.choice(moves)
                else:
                    best_move = possible_scores.index(max(possible_scores))
                self.make_move(best_move, room)
                self.prev_move = best_move
                self.limit += 1
                return True

## Runing the agent

In [9]:
my_agent = cleaner_agent()
my_env = room_environment(6, 6)

my_env.print_room()
print("Dirt count in the map: ", my_env.dirt_count())

0 0 -1 0 0 0 
-1 0 0 0 0 0 
0 0 0 0 0 -1 
0 0 0 0 0 0 
0 1 0 -1 0 0 
0 0 0 0 0 0 
Dirt count in the map:  1


In [10]:
status = True
while(status):
    status = my_agent.move(my_env)
    my_agent.get_pos()

I am at: 0 0  and picked up:  0  dirt.
I am at: 0 0  and picked up:  0  dirt.
I am at: 0 0  and picked up:  0  dirt.
I am at: 0 0  and picked up:  0  dirt.
I am at: 1 0  and picked up:  0  dirt.
I am at: 1 1  and picked up:  0  dirt.
I am at: 1 2  and picked up:  0  dirt.
I am at: 1 3  and picked up:  0  dirt.
I am at: 1 4  and picked up:  1  dirt.
Collected All the Dirst, Job Done...
I am at: 1 4  and picked up:  1  dirt.


In [11]:
my_env.print_room()

0 0 -1 0 0 0 
-1 0 0 0 0 0 
0 0 0 0 0 -1 
0 0 0 0 0 0 
0 0 0 -1 0 0 
0 0 0 0 0 0 
